# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [58]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [59]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kailua-kona,19.6406,-155.9956,80.51,78,40,3.44,US,Mon Dec 11 13:49:09 2023
1,1,grytviken,-54.2811,-36.5092,39.99,76,96,12.46,GS,Mon Dec 11 13:49:10 2023
2,2,bethel,41.3712,-73.4140,43.52,74,100,5.75,US,Mon Dec 11 13:49:10 2023
3,3,tromso,69.6496,18.9570,26.62,80,75,9.22,NO,Mon Dec 11 13:49:11 2023
4,4,albany,42.6001,-73.9662,40.37,75,10,11.01,US,Mon Dec 11 13:45:44 2023


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [60]:
%%capture --no-display

# Configure the map plot
map_plot_1= city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    scale =1,
    size="Humidity",
   color="Country",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500)
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [66]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_data_df = city_data_df[(city_data_df["Max Temp"]<100)&(city_data_df["Country"]=="BR")&(city_data_df["Cloudiness"]<80)]

# Drop any rows with null values
vacation_data_df = vacation_data_df.dropna()

# Display sample data
vacation_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
79,79,arraial do cabo,-22.9661,-42.0278,75.16,88,75,11.50,BR,Mon Dec 11 13:49:42 2023
102,102,touros,-5.1989,-35.4608,83.46,64,0,14.52,BR,Mon Dec 11 13:49:52 2023
150,150,sao gabriel da cachoeira,-0.1303,-67.0892,82.20,89,75,9.22,BR,Mon Dec 11 13:50:12 2023
190,190,piripiri,-4.2733,-41.7769,97.02,28,25,6.89,BR,Mon Dec 11 13:50:31 2023
226,226,carutapera,-1.1950,-46.0200,85.86,63,55,10.85,BR,Mon Dec 11 13:50:46 2023
237,237,barra,-11.0894,-43.1417,92.66,33,37,4.68,BR,Mon Dec 11 13:50:50 2023
248,248,santana,-0.0583,-51.1817,91.58,55,40,14.97,BR,Mon Dec 11 13:50:54 2023
272,272,ilhabela,-23.7781,-45.3581,75.88,85,64,6.33,BR,Mon Dec 11 13:51:04 2023
294,294,cangucu,-31.3950,-52.6756,78.01,69,74,8.61,BR,Mon Dec 11 13:51:14 2023
313,313,fortaleza,-3.7227,-38.5247,88.07,64,20,18.41,BR,Mon Dec 11 13:51:25 2023


### Step 3: Create a new DataFrame called `hotel_df`.

In [67]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=vacation_data_df[["City", "Country", "Lat","Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:,"Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
79,arraial do cabo,BR,-22.9661,-42.0278,88,
102,touros,BR,-5.1989,-35.4608,64,
150,sao gabriel da cachoeira,BR,-0.1303,-67.0892,89,
190,piripiri,BR,-4.2733,-41.7769,28,
226,carutapera,BR,-1.1950,-46.0200,63,
237,barra,BR,-11.0894,-43.1417,33,
248,santana,BR,-0.0583,-51.1817,55,
272,ilhabela,BR,-23.7781,-45.3581,85,
294,cangucu,BR,-31.3950,-52.6756,69,
313,fortaleza,BR,-3.7227,-38.5247,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [69]:
# Set parameters to search for a hotel
radius = 10000
api_key = "a0da719002cd48188e13bc8eb114fc3c"
params = {
    "categories":"accommodation.hotel",
    "apiKey":api_key,
    "limit":20
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)

    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
arraial do cabo - nearest hotel: Pousada Vida Boa Praia
touros - nearest hotel: Siri Paraíso Hotel
sao gabriel da cachoeira - nearest hotel: Casa de Hóspedes do CFRN/5ºBIS
piripiri - nearest hotel: Passion
carutapera - nearest hotel: No hotel found
barra - nearest hotel: No hotel found
santana - nearest hotel: Hotel Teodoro
ilhabela - nearest hotel: Hotel do Sol Executive
cangucu - nearest hotel: Hotel Tiaraju
fortaleza - nearest hotel: Hotel Amuarama
cidreira - nearest hotel: Lagoa Country Club
acarau - nearest hotel: No hotel found
piacabucu - nearest hotel: O Leão
vila velha - nearest hotel: Hostess Hotel
castro - nearest hotel: Hotel Fazenda das 100 Árvores
areia branca - nearest hotel: Areia Branca Apart Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
79,arraial do cabo,BR,-22.9661,-42.0278,88,Pousada Vida Boa Praia
102,touros,BR,-5.1989,-35.4608,64,Siri Paraíso Hotel
150,sao gabriel da cachoeira,BR,-0.1303,-67.0892,89,Casa de Hóspedes do CFRN/5ºBIS
190,piripiri,BR,-4.2733,-41.7769,28,Passion
226,carutapera,BR,-1.1950,-46.0200,63,No hotel found
237,barra,BR,-11.0894,-43.1417,33,No hotel found
248,santana,BR,-0.0583,-51.1817,55,Hotel Teodoro
272,ilhabela,BR,-23.7781,-45.3581,85,Hotel do Sol Executive
294,cangucu,BR,-31.3950,-52.6756,69,Hotel Tiaraju
313,fortaleza,BR,-3.7227,-38.5247,64,Hotel Amuarama


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [71]:
%%capture --no-display

# Configure the map plot
map_plot_hotel= hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    scale =1,
    size="Humidity",
   color="Country",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols=["Hotel Name", "Country"])

# Display the map
map_plot_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,Hotel Name)